# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Settings

In [1]:
# Control the notebook execution settings from here
# This way you can use the "Run Cells" functionality for one-click execution, 
# but only execute the parts you want

training_file = 'train.p'
testing_file = 'test.p'
flags = {
    'show_plots': False, 
    'generate_color_images': False,
    'generate_grayscale_images': False,
    'train_network': True,
    'evaluate_test_accuracy': True
}

# HyperParameters for Training
EPOCHS = 5
BATCH_SIZE = 128
MU = 0
SIGMA = 0.1
LEARNING_RATE = .001
CONV_KEEP_PROB = 1
KEEP_PROB = 1

In [2]:
# Run all imports
import pickle
import random
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import KFold
%matplotlib inline

---
## Step 0: Load The Data

In [3]:
# Load pickled data
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [4]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = (X_train.shape[1], X_train.shape[2])

# TODO: How many unique classes/labels there are in the dataset.
n_classes = max(y_train) + 1

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32)
Number of classes = 43


Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [5]:
# How many of each sign?
if flags['show_plots']:
    plt.hist(y_train, bins=range(1,n_classes+1))
    plt.figure()
    plt.hist(y_test, bins=range(1, n_classes+1))
    print()

In [6]:
if flags['show_plots']:
    for i in range(5):
        plt.figure(figsize=(1,1))
        plt.imshow(random.choice(X_train))

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

#### Generating Additional and Alternate Data

In [7]:
# Generate color images so that each label has atleast 1500 examples

import random
import cv2

def generate_color_image(label):   
    indexes = [x[0] for x in enumerate(y_train) if x[1] == label]
    index = random.choice(indexes)
    image = X_train[index]
    rotate = random.randint(-10, 10)
    scale = random.random() * .2 + 1
    image_center = (16, 16)
    rot_mat = cv2.getRotationMatrix2D(image_center,rotate, scale)
    result = cv2.warpAffine(image, rot_mat, (32, 32))
    return result

if flags['generate_color_images']:
    generated = []
    generated_labels = []
    bins = np.bincount(y_train)
    for label, bin in enumerate(bins):
        need = 1500 - bin if bin < 1500 else 0
        print("Label {0} needs {1} images".format(label, need))
        for i in range(need):
            generated.append(generate_image(label))
            generated_labels.append(label)
    
    with open('generated.p', 'wb') as f:
        pickle.dump({ 'features': generated, 'labels': generated_labels }, f)

In [8]:
# Convert RGB images to grayscale
def grayscale(images):
    converted = []
    for image in images:
        converted.append(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
    return np.array(converted)

if flags['generate_grayscale_images']:
    train_grayscale = grayscale(X_train)
    test_grayscale  = grayscale(X_test)
    train_data = {
        'features': train_grayscale, 
        'labels': y_train
    }
    test_data = {
        'features': test_grayscale,
        'labels': y_test
    }
    with open('train_grayscale.p' ,'wb') as f:
        pickle.dump(train_data, f)
    with open('test_grayscale.p', 'wb') as f:
        pickle.dump(test_data, f)

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

In [9]:
X_train, y_train = sklearn.utils.shuffle(X_train, y_train)

# kf = KFold(n_splits=5)
# for sub_train, val in kf.split(X_train):
#     X_sub_train, X_val, y_sub_train, y_val = X_train[sub_train], X_train[val], y_train[sub_train], y_train[val]

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [10]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [11]:
from tensorflow.contrib.layers import flatten

def maxpool(input_layer):
    ksize = [1, 2, 2, 1]
    strides = [1, 2, 2, 1]
    padding = 'VALID'
    return tf.nn.max_pool(input_layer, ksize, strides, padding)
    
def conv_layer(input_layer, filter_shape, strides=1, padding='VALID'):
    weights = tf.Variable(tf.truncated_normal(filter_shape, mean=MU, stddev=SIGMA))
    biases = tf.Variable(tf.zeros(filter_shape[3]))
    strides = [1,strides,strides,1]
    return tf.nn.conv2d(input_layer, weights, strides, padding) + biases

def fully_connected_layer(input_layer, shape):
    weights = tf.Variable(tf.truncated_normal(shape, mean=MU, stddev=SIGMA))
    biases = tf.Variable(tf.zeros(shape[1]))
    return tf.matmul(input_layer, weights) + biases

def activation_layer(input_layer):
    return tf.sigmoid(input_layer)
    
def LeNet(x, keep_prob, conv_keep_prob): 
    # Output = 14 x 14 x 25
    conv1 = conv_layer(x, [5,5,3,25])
    conv1_activated = activation_layer(conv1)
    conv1_pooled = maxpool(conv1_activated)
    conv1_dropped = tf.nn.dropout(conv1_pooled, conv_keep_prob)
    
    # Output = 5 x 5 x 150
    conv2 = conv_layer(conv1_dropped, [5,5,25,150])
    conv2_activated = activation_layer(conv2)
    conv2_pooled = maxpool(conv2_activated)
    conv2_dropped = tf.nn.dropout(conv2_pooled, conv_keep_prob)
    
    fc1_input = flatten(conv2_pooled)
    fc1_output = fully_connected_layer(fc1_input, (3750, 500))
    fc1_activated = activation_layer(fc1_output)
    fc1_dropped = tf.nn.dropout(fc1_activated, keep_prob)
    
    fc2_output = fully_connected_layer(fc1_dropped, (500, 250))
    fc2_activated = activation_layer(fc2_output)
    fc2_dropped = tf.nn.dropout(fc2_activated, keep_prob)
    
    logits = fully_connected_layer(fc2_dropped, (250, 43))
    
    return logits

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1, conv_keep_prob: 1})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

# Setup Variables
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
conv_keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)

# Training Network
logits = LeNet(x, keep_prob, conv_keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

In [12]:
if flags['train_network']: 
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        print("Training...")
        print()
        for i in range(EPOCHS):
            print("EPOCH {} ...".format(i+1))
            train_accuracies = []
            val_accuracies = []
            X_train, y_train = sklearn.utils.shuffle(X_train, y_train)
            kf = sklearn.model_selection.KFold(n_splits=3)
            for j, (sub_train, val) in enumerate(kf.split(X_train)):
                X_sub_train, X_val, y_sub_train, y_val = X_train[sub_train], X_train[val], y_train[sub_train], y_train[val]
                num_examples = len(X_sub_train)
                for offset in range(0, num_examples, BATCH_SIZE):
                    end = offset + BATCH_SIZE
                    batch_x, batch_y = X_sub_train[offset:end], y_sub_train[offset:end]
                    sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: KEEP_PROB, conv_keep_prob: CONV_KEEP_PROB})
                train_accuracy = evaluate(X_sub_train, y_sub_train)
                val_accuracy = evaluate(X_val, y_val)
                train_accuracies.append(train_accuracy)
                val_accuracies.append(val_accuracy)
                print("CV {}".format(j+1))
                print("Train Accuracy = {:.3f}".format(train_accuracy))
                print("Validation Accuracy = {:.3f}".format(val_accuracy))
            print("Averaged CVs: Train Accuracy = {:.3f}".format(np.mean(train_accuracies)))
            print("Averaged CVs: Validation Accuracy = {:.3f}".format(np.mean(val_accuracies)))
            print()

        saver.save(sess, 'lenet')
        print("Model saved")

Training...

EPOCH 1 ...
CV 1
Train Accuracy = 0.539
Validation Accuracy = 0.536
CV 2
Train Accuracy = 0.950
Validation Accuracy = 0.947
CV 3
Train Accuracy = 0.986
Validation Accuracy = 0.984
Averaged CVs: Train Accuracy = 0.825
Averaged CVs: Validation Accuracy = 0.823

EPOCH 2 ...
CV 1
Train Accuracy = 0.993
Validation Accuracy = 0.990
CV 2
Train Accuracy = 0.995
Validation Accuracy = 0.993
CV 3
Train Accuracy = 0.997
Validation Accuracy = 0.997
Averaged CVs: Train Accuracy = 0.995
Averaged CVs: Validation Accuracy = 0.993

EPOCH 3 ...
CV 1
Train Accuracy = 0.999
Validation Accuracy = 0.997
CV 2
Train Accuracy = 0.999
Validation Accuracy = 0.999
CV 3
Train Accuracy = 0.997
Validation Accuracy = 0.996
Averaged CVs: Train Accuracy = 0.998
Averaged CVs: Validation Accuracy = 0.997

EPOCH 4 ...
CV 1
Train Accuracy = 0.999
Validation Accuracy = 0.999
CV 2
Train Accuracy = 0.996
Validation Accuracy = 0.994
CV 3
Train Accuracy = 0.997
Validation Accuracy = 0.996
Averaged CVs: Train Accurac

In [13]:
if flags['evaluate_test_accuracy']:
    with tf.Session() as sess:
        saver.restore(sess, './lenet')     
        test_accuracy = evaluate(X_test, y_test)
        print("Test Accuracy: {:.3f}".format(test_accuracy))

Test Accuracy: 0.965


### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [14]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [15]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [16]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.